# Unrestricted Hartee-Fock

The Hartree-Fock method we covered last week is restricted to closed-shell systems in which you have the same amount of $\alpha$ and $\beta$ electrons. This means many systems are not treatable by Restricted Hatree-Fock, such as radicals, bond breaking, and paramagnetic systems.
 
For Unrestricted Hartree-Fock (UHF), the main difference is that a set of coupled equations are introduced. There are equations for the $\alpha$ electrons and equations for the $\beta$ electrons. Note there is another method for treating open shell systems called Restricted Open-shell Hartree-Fock (ROHF) that we don't cover here.

## Some useful resources:
 - Szabo and Ostlund Chapter 3 (for algorithm see section 3.8)
 - [Notes by David Sherrill](http://vergil.chemistry.gatech.edu/notes/hf-intro/hf-intro.html)
 - [Psi4Numpy SCF page](https://github.com/psi4/psi4numpy/tree/master/Tutorials/03_Hartree-Fock)

## Imports

In [1]:
import numpy as np
import scipy.linalg as spla
import pyscf
from pyscf import gto, scf
import time

## The UHF algorithm from Szabo and Ostlund:
 1. Specify a molecule (coordinates $\{R_A\}$, atomic numbers $\{Z_A\}$, number of $\alpha $ electrons $N_\alpha$ and number of $\beta$ electrons $N_\beta$) and atomic orbital basis $\{\phi_\mu\}$.
 2. Calculate molecular integrals over AOs ( overlap $S_{\mu\nu}$, core Hamiltonian $H^{\mathrm{core}}_{\mu\nu}$, and 2  electron integrals $(\mu \nu | \lambda \sigma)$ ).
 3. Make a guess at the original density matrix $P^\alpha$ and $P^\beta$. Where
 $P^\alpha + P^\beta = P^\mathrm{Total}$
 4. Calculate the intermediate matrix $G^\alpha$ using the density matrix $P^\alpha$ and the two electron integrals $(\mu \nu | \lambda \sigma)$
 5. Calculate the intermediate matrix $G^\beta$ using the density matrix $P^\beta$ and the two electron integrals $(\mu \nu | \lambda \sigma)$
 6. Construct the two Fock matrices, one for the $\alpha$ electrons $F^\alpha$ and one for the $\beta$ electrons $F^\beta$. Each is composed from the core Hamiltonian $H^{\mathrm{core}}_{\mu\nu}$ and the respective intermediate matrix $G$. 
 7. Solve the generalized eigenvalue problem for each of the Fock matrices and the overlap matrix $S$ to get orbital energies $\epsilon$ and molecular orbitals $C^\alpha$ and $C^\beta$. \**  
 8. Form a new guess at the density matrices $P^{\mathrm{Total}}$, $P^\alpha$ and $P^\beta$ using $C^\alpha$ and $C^\beta$, respectively.
 9. Check for convergence. (Are the changes in energy and/or density smaller than some threshold?) If not, return to step 4.
 10. If converged, use the molecular orbitals $C$, density matrices $P$, and Fock matrix $F$ to calculate observables like the total energy, etc.
 
\** This can also be solved with the method of orthogonalizing the atomic orbitals as shown in the basic Hartree-Fock approach 

# STEP 1 : Specify the molecule

Note: Modifying charge and multiplicity in water in order to demonstrate UHF capability. If charge is 0 and multiplicity is 1, UHF will be the same as RHF for our water example.

In [2]:
# start timer
start_time = time.time()
# define molecule
mol = pyscf.gto.M(
    atom="""O 0.0000000 0.0000000 0.0000000;
    H 0.7569685 0.0000000 -0.5858752;
    H -0.7569685 0.0000000 -0.5858752""",
    basis='sto-3g',
    unit = "Ang",
    verbose=0,
    symmetry=False,
    spin = 1,
    charge = -1
)
# get number of atomic orbitals
num_ao = mol.nao_nr()
# get number of electrons
num_elec_alpha, num_elec_beta = mol.nelec
num_elec = num_elec_alpha + num_elec_beta
# get nuclear repulsion energy
E_nuc = mol.energy_nuc()

# STEP 2 : Calculate molecular integrals 

Overlap 

$$ S_{\mu\nu} = (\mu|\nu) = \int dr \phi^*_{\mu}(r) \phi_{\nu}(r) $$

Kinetic

$$ T_{\mu\nu} = (\mu\left|-\frac{\nabla}{2}\right|\nu) = \int dr \phi^*_{\mu}(r) \left(-\frac{\nabla}{2}\right) \phi_{\nu}(r) $$

Nuclear Attraction

$$ V_{\mu\nu} = (\mu|r^{-1}|\nu) = \int dr \phi^*_{\mu}(r) r^{-1} \phi_{\nu}(r) $$

Form Core Hamiltonian

$$ H^\mathrm{core} = T + V $$

Two electron integrals

$$ (\mu\nu|\lambda\sigma) = \int dr_1 dr_2 \phi^*_{\mu}(r_1) \phi_{\nu}(r_1) r_{12}^{-1} \phi_{\lambda}(r_2) \phi_{\sigma}(r_2) $$


In [3]:
# calculate overlap integrals
S = mol.intor('cint1e_ovlp_sph')
# calculate kinetic energy integrals
T = mol.intor('cint1e_kin_sph')
# calculate nuclear attraction integrals
V = mol.intor('cint1e_nuc_sph')
# form core Hamiltonian
H = T + V
# calculate two electron integrals
eri = mol.intor('cint2e_sph',aosym='s8')
# since we are using the 8 fold symmetry of the 2 electron integrals
# the functions below will help us when accessing elements
__idx2_cache = {}
def idx2(i, j):
    if (i, j) in __idx2_cache:
        return __idx2_cache[i, j]
    elif i >= j:
        __idx2_cache[i, j] = int(i*(i+1)/2+j)
    else:
        __idx2_cache[i, j] = int(j*(j+1)/2+i)
    return __idx2_cache[i, j]
def idx4(i, j, k, l):
    return idx2(idx2(i, j), idx2(k, l))

# STEP 3 : Core Guess

In [4]:
# AO orthogonalization matrix 
A = spla.fractional_matrix_power(S, -0.5)

# Solve the generalized eigenvalue problem
E_orbitals, C = spla.eigh(H,S)

# Compute initial density matrix
D_alpha = np.zeros((num_ao,num_ao))
D_beta = np.zeros((num_ao,num_ao))
for i in range(num_ao):
    for j in range(num_ao):
        for k in range(num_elec_alpha):
            D_alpha[i,j] +=  C[i,k] * C[j,k]
        for k in range(num_elec_beta):
            D_beta[i,j] +=  C[i,k] * C[j,k]

D_total = D_alpha + D_beta

# STEP 4: DIIS
[DIIS Theory Overview](https://github.com/shivupa/QMMM_study_group/blob/master/03_advanced_SCF/diis_pyscf.ipynb)
### Steps in DIIS Function
1. Build B matrix
2. Solve the Pulay equation
3. Build the DIIS Fock matrix

In [5]:
def diis(F_list, diis_res):
    # Build B matrix 
    dim_B = len(F_list) + 1
    B = np.empty((dim_B, dim_B))
    B[-1, :] = -1
    B[:, -1] = -1
    B[-1, -1] = 0
    for i in range(len(F_list)):
        for j in range(len(F_list)):
            B[i, j] = np.einsum('ij,ij->', diis_res[i], diis_res[j])
    
    # Right hand side of Pulay eqn
    right = np.zeros(dim_B)
    right[-1] = -1

    # Solve Pulay for coeffs
    cn = np.linalg.solve(B, right)
    
    # Build DIIS Fock
    F_diis = np.zeros_like(F_list[0])
    for x in range(cn.shape[0] - 1):
        F_diis += cn[x] * F_list[x]
        
    return F_diis

# STEPS 5 - 9 : SCF loop

 5. Calculate the intermediate matrix $G$ using the density matrix $P$ and the two electron integrals $(\mu \nu | \lambda \sigma)$.
 
 $$G^\alpha_{\mu\nu} = \sum_{\lambda\sigma}^{\mathrm{num\_ao}} P^T_{\lambda \sigma}(\mu\nu|\lambda\sigma)-P_{\lambda \sigma}^\alpha(\mu\lambda|\nu\sigma)$$
 
  $$G^\beta_{\mu\nu} = \sum_{\lambda\sigma}^{\mathrm{num\_ao}} P^T_{\lambda \sigma}(\mu\nu|\lambda\sigma)-P_{\lambda \sigma}^\beta(\mu\lambda|\nu\sigma)]$$ 
 
 6. Construct the Fock matrix $F$ from the core hamiltonian $H^{\mathrm{core}}_{\mu\nu}$ and the intermediate matrix $G$. 
 
 $$F^\alpha\ =\ H^{\mathrm{core}}\ + G^\alpha $$
 
  $$F^\beta\ =\ H^{\mathrm{core}}\ + G^\beta $$
 
 7. Solve the generalized eigenvalue problem using the Fock matrix $F$ and the overlap matrix $S$ to get orbital energies $\epsilon$ and molecular orbitals.
 
 $$F^\alpha C^\alpha\ =\ SC^\alpha\epsilon^\alpha$$
 $$F^\beta C^\beta\ =\ SC^\beta\epsilon^\beta$$
 
 8. Form a new guess at the density matrix $P$ using $C$.
 
 $$ P^\alpha_{\mu\nu} = \sum_{i}^{\mathrm{N_\alpha}} C^\alpha_{\mu i} C^\alpha_{\nu i} $$
  $$ P^\beta_{\mu\nu} = \sum_{i}^{\mathrm{N_\beta}} C^\beta_{\mu i} C^\beta_{\nu i} $$
  $$ P^{\mathrm{Total}} = P^\alpha + P^\beta $$
 9. Check for convergence. (Are the changes in energy and density smaller than some threshold?) If not, return to step 5.
 
 $$ E_{\mathrm{elec}}  = \sum^{\mathrm{num\_ao}}_{\mu\nu} P^T_{\mu\nu} H^\mathrm{core}_{\mu\nu} + P^\alpha_{\mu\nu}F^\alpha_{\mu\nu} + P^\beta_{\mu\nu}F^\beta_{\mu\nu}  $$
 $$ \Delta E = E_{\mathrm{new}} - E_{\mathrm{old}} $$
 $$ |\Delta P| = \left[ \sum^{\mathrm{num\_ao}}_{\mu\nu} [P^{\mathrm{Total new}}_{\mu\nu} - P_{\mu\nu}^{\mathrm{Total old}}]^2 \right]^{1/2}$$
 

In [6]:
# 2 helper functions for printing during SCF
def print_start_iterations():
    print("{:^79}".format("{:>4}  {:>11}  {:>11}  {:>11}  {:>11}".format("Iter", "Time(s)", "RMSC DM", "delta E", "E_elec")))
    print("{:^79}".format("{:>4}  {:>11}  {:>11}  {:>11}  {:>11}".format("****", "*******", "*******", "*******", "******")))
def print_iteration(iteration_num, iteration_start_time, iteration_end_time, iteration_rmsc_dm, iteration_E_diff, E_elec):
    print("{:^79}".format("{:>4d}  {:>11f}  {:>.5E}  {:>.5E}  {:>11f}".format(iteration_num, iteration_end_time - iteration_start_time, iteration_rmsc_dm, iteration_E_diff, E_elec)))

# Set stopping criteria
iteration_max = 100
convergence_E = 1e-9
convergence_DM = 1e-5
# Loop variables
iteration_num = 0
E_total = 0
E_elec = 0.0
iteration_E_diff = 0.0
iteration_rmsc_dm = 0.0
converged = False
exceeded_iterations = False

In [7]:
# Trial & Residual vector lists
F_list_alpha = []
F_list_beta = []
DIIS_resid_alpha = []
DIIS_resid_beta = []

print("{:^79}".format('=====> Starting SCF Iterations <=====\n'))
print_start_iterations()
while (not converged and not exceeded_iterations):
    # Store last iteration and increment counters
    iteration_start_time = time.time()
    iteration_num += 1
    E_elec_last = E_elec
    D_total_last = np.copy(D_total)
    
    # Form G matrix
    G_alpha = np.zeros((num_ao,num_ao))
    G_beta = np.zeros((num_ao,num_ao))
    for i in range(num_ao):
        for j in range(num_ao):
            for k in range(num_ao):
                for l in range(num_ao):
                    G_alpha[i,j] += D_alpha[k,l] * ((2.0*(eri[idx4(i,j,k,l)])) - (eri[idx4(i,k,j,l)]))
                    G_beta[i,j] += D_beta[k,l] * ((2.0*(eri[idx4(i,j,k,l)])) - (eri[idx4(i,k,j,l)]))
    
    # Build fock matrices
    F_alpha  = H + G_alpha
    F_beta  = H + G_beta
    
    # Calculate electronic energy
    E_elec = 0.5 * np.sum(np.multiply((D_total), H) + np.multiply(D_alpha, F_alpha) + np.multiply(D_beta, F_beta))
    
    # Build the DIIS AO gradient
    diis_r_alpha = A.T @ (F_alpha @ D_alpha @ S - S @ D_alpha @ F_alpha) @ A
    diis_r_beta = A.T @ (F_beta @ D_beta @ S - S @ D_beta @ F_beta) @ A
    
    # DIIS RMS
    diis_rms = (np.mean(diis_r_alpha**2)**0.5 + np.mean(diis_r_beta**2)**0.5) * 0.5

    # Append lists
    F_list_alpha.append(F_alpha)
    F_list_beta.append(F_beta)
    DIIS_resid_alpha.append(diis_r_alpha)
    DIIS_resid_beta.append(diis_r_beta)

    if iteration_num >=2:
        # Preform DIIS to get Fock Matrix
        F_alpha = diis(F_list_alpha, DIIS_resid_alpha)
        F_beta = diis(F_list_beta, DIIS_resid_beta)
    
    # Compute new guess with F DIIS
    E_orbitals_alpha, C_alpha = spla.eigh(F_alpha,S)
    E_orbitals_beta, C_beta = spla.eigh(F_beta,S)
    D_alpha = np.zeros((num_ao,num_ao))
    D_beta = np.zeros((num_ao,num_ao))
    
    for i in range(num_ao):
        for j in range(num_ao):
            for k in range(num_elec_alpha):
                D_alpha[i,j] +=  C_alpha[i,k] * C_alpha[j,k]
            for k in range(num_elec_beta):
                D_beta[i,j] +=  C_beta[i,k] * C_beta[j,k]
                
    D_total = D_alpha + D_beta
    
    # Calculate energy change of iteration
    iteration_E_diff = np.abs(E_elec - E_elec_last)
    # RMS change of density matrix
    iteration_rmsc_dm = np.sqrt(np.sum((D_total - D_total_last)**2))
    iteration_end_time = time.time()
    print_iteration(iteration_num, iteration_start_time, iteration_end_time, iteration_rmsc_dm, iteration_E_diff, E_elec)
    if(np.abs(iteration_E_diff) < convergence_E and iteration_rmsc_dm < convergence_DM): 
        converged = True
        print('\n',"{:^79}".format('=====> SCF Converged <=====\n'))
        # calculate total energy
        E_total = E_elec + E_nuc
        print("{:^79}".format("Total Energy : {:>11f}".format(E_total)))
    if(iteration_num == iteration_max):
        exceeded_iterations = True
        print("{:^79}".format('=====> SCF Exceded Max Iterations <=====\n'))

                    =====> Starting SCF Iterations <=====
                     
           Iter      Time(s)      RMSC DM      delta E       E_elec            
           ****      *******      *******      *******       ******            
              1     0.015874  2.95622E+00  8.17590E+01   -81.758995            
              2     0.014794  1.14565E-01  1.37090E+00   -83.129897            
              3     0.016064  1.79605E-02  1.59234E-02   -83.145820            
              4     0.014538  6.58871E-03  6.72414E-04   -83.146492            
              5     0.014321  2.55353E-03  3.12590E-05   -83.146524            
              6     0.014714  2.41061E-04  9.43047E-07   -83.146525            
              7     0.015249  2.99870E-07  9.44209E-09   -83.146525            
              8     0.015318  8.88629E-09  7.10543E-14   -83.146525            

                          =====> SCF Converged <=====
                          
                          Total Energy